In [2]:
import pandas as pd
from datetime import datetime as dt
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import json

# Fetch the service account key JSON file contents
cred = credentials.Certificate('../Secrets/chicagocrimeanalysis-a24e2-firebase-adminsdk-7b0qc-5abe0f5d88.json')

# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://chicagocrimeanalysis-a24e2-default-rtdb.firebaseio.com'
})



In [3]:
df = pd.read_csv("../Data/Crimes2001_to_Present.csv")

In [4]:
df["Month"] = df["Date"].apply(lambda x: dt.strptime(x, "%m/%d/%Y %I:%M:%S %p").month)

In [5]:
years = [i for i in range(2022, 2024)]
months = [i for i in range(1, 13)]
for year in years:
    for month in months:
       partial_df = df[(df["Year"] == year) & (df["Month"] == month)].copy()
       partial_df.dropna(inplace=True)  # Modify the copied DataFrame
       data_dict = partial_df.to_dict(orient='split')["data"]
       ref = db.reference(f"/{year}/{month}/")
       ref.set(data_dict)

In [6]:
#creatying one big json file
partial_df = df[(df["Year"] >= 2017) & (df["Month"] <= 2023)].copy()
partial_df.dropna(inplace=True)  # Modify the copied DataFrame
partial_df.to_json("../Data/2017-2023.json", orient='split')